In [1]:
import preprocess
import trader
import helper
import pymysql
import matplotlib.pyplot as plt

In [2]:
pr = preprocess.Preprocess()

In [3]:
order_tables = pr.get_order_tables_names()
trans_tables = pr.get_trans_tables_names()

In [4]:
order_df = pr.get_table_df(order_tables[5])
trans_df = pr.get_table_df(trans_tables[5])

In [5]:
sobi = preprocess.get_sobi(order_df)
vpin = preprocess.get_trans_vpin(trans_df, max_capa=1000)

In [6]:
trans_df.head()

,time,price,amount
0,2022-08-09 09:00:16,3245,6164
1,2022-08-09 09:00:16,3255,1
2,2022-08-09 09:00:20,3265,25
3,2022-08-09 09:00:20,3270,1
4,2022-08-09 09:00:20,3270,409


In [7]:
vpin[['trans_idx', 'vpin']].head()

,trans_idx,vpin
0,8,1.000000
1,19,0.058399
2,21,1.000000
3,24,0.793722
4,33,0.705922


In [8]:
sobi[['time', 'mid_vwap']].head()

,time,mid_vwap
0,2022-08-09 08:57:52,3239.950748
1,2022-08-09 08:59:09,3239.859271
2,2022-08-09 08:59:09,3239.859271
3,2022-08-09 08:59:12,3240.570859
4,2022-08-09 08:59:18,3240.570859


In [9]:
def get_signal(vpin):
    signal = vpin[vpin['vpin'] > .3]['trans_idx']
    return signal

def get_signals_2(sobi):
    signal = sobi[sobi['mid_vwap'] - sobi['mid_price'] > 0].index
    signal = list(signal)
    return signal

In [10]:
get_signal(vpin)
get_signals_2(sobi)

[12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 43,
 45,
 47,
 80,
 81,
 82,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 187,
 203,
 204,
 205,
 206,
 207,
 209,
 210,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 253,
 254,
 255,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 283,
 284,
 296,
 297,
 298,
 299,
 300,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 344,
 345,
 346,
 347,
 348,
 349,
 354,
 355,
 356,
 35

In [11]:
idxs = vpin[vpin['vpin'] > .7]['trans_idx']
trans_df.loc[idxs, 'time']

8     2022-08-09 09:00:26
21    2022-08-09 09:04:35
24    2022-08-09 09:06:15
33    2022-08-09 09:08:57
36    2022-08-09 09:09:02
              ...        
840   2022-08-09 14:38:10
861   2022-08-09 14:50:57
922   2022-08-09 15:12:03
925   2022-08-09 15:12:17
952   2022-08-09 15:30:07
Name: time, Length: 88, dtype: datetime64[ns]

In [12]:
def trade():
    trans_df
    order_df
    sobi
    vpin

    signals = get_signal(vpin)
    total_orders = []
    
    order_tables[10]
    trans_tables[15]

    for sig in signals:
        # print('index: ', sig, 'price: ', abs(trans_df.iloc[sig]['price']))
        order = trader.Order(0, trans_tables[15], abs(trans_df.iloc[sig]['price']), '', 2)

        total_orders.append(order)
        to_remove = []
        for o_id, o in enumerate(total_orders):

            result = o.trailing_stop(trans_df.iloc[sig]['price']) # bid
            if result == 'stop':
                # record_result(o)
                print(abs(o.in_price), abs(o.out_price))
                to_remove.append(o_id)
            else:
                # hold
                pass

        total_orders = [o for i, o in enumerate(total_orders) if i not in to_remove]
    
    for o in total_orders:
        # print(abs(o.in_price), abs(o.out_price))
        print(o.in_price, o.trail)


def trade2():
    order_df
    signals = get_signals_2(sobi)

    for sig in signals:
        print(sig, order_df.loc[sig, 'ask_1'], order_df.loc[sig, 'bid_1'])

In [13]:
trade()

3280 3285 up
3280 3290 up
3285 3290 up
3280 3290 up
3280 3295 up
3285 3295 up
3280 3295 up
3290 3295 up
3260 3275 up
3260 3275 up
3260 3275 up
3260 3270 up
3255 3270 up
3255 3270 up
3255 3270 up
3260 3275 up
3255 3275 up
3255 3275 up
3255 3275 up
3270 3275 up
3260 3280 up
3260 3280 up
3260 3280 up
3275 3280 up
3260 3280 up
3255 3280 up
3255 3280 up
3255 3280 up
3270 3280 up
3275 3280 up
3275 3280 up
3280 3285 up
3280 3285 up
3260 3285 up
3260 3285 up
3260 3285 up
3275 3285 up
3260 3285 up
3255 3285 up
3255 3285 up
3255 3285 up
3270 3285 up
3275 3285 up
3275 3285 up
3280 3285 up
3280 3285 up
3280 3295 up
3280 3295 up
3260 3295 up
3260 3295 up
3260 3295 up
3275 3295 up
3260 3295 up
3255 3295 up
3255 3295 up
3255 3295 up
3270 3295 up
3275 3295 up
3275 3295 up
3280 3295 up
3280 3295 up
3285 3295 up
3280 3300 up
3285 3300 up
3280 3300 up
3290 3300 up
3295 3300 up
3280 3300 up
3280 3300 up
3260 3300 up
3260 3300 up
3260 3300 up
3275 3300 up
3260 3300 up
3255 3300 up
3255 3300 up
3255 3300 up

In [8]:
d = {'a':1, 'b':2}
d.popitem()

('b', 2)